In [75]:
using LinearAlgebra;

In [288]:
function csminwel(fcn, x0, H0, grad, crit, nit; kwargs...)
    # Initialize variables
    nx = length(x0)
    x = copy(x0)
    H = Matrix(H0)  # Ensure H is a dense matrix
    NumGrad = isnothing(grad)
    f = fcn(x; kwargs...)

    if f > 1e50
        error("Bad initial parameter.")
    end

    # Compute initial gradient
    g, badg = NumGrad ? numerical_gradient(fcn, x; kwargs...) : grad(x; kwargs...)

    itct = 0
    fcount = 0
    done = false

    while !done
        println("-----------------")
        println("Iteration: $itct")
        println("Function value: $f")
        println("x: $x")

        itct += 1

        f1, x1, fc, retcode1 = csminit(fcn, x, f, g, badg, H; kwargs...)
        fcount += fc
        print("$retcode1\n")
        g1, badg1 = NumGrad ? numerical_gradient(fcn, x1; kwargs...) : grad(x1; kwargs...)


        if retcode1 != 1

            if retcode1 in [2, 4]
                wall1 = true
            else
                wall1 = badg1
            end

            if wall1 && nx > 1
                println("Cliff detected. Perturbing search direction.")
                H_cliff = H + Diagonal(rand(nx) .* diag(H))
                f2, x2, fc, retcode2 = csminit(fcn, x, f, g, badg, H_cliff; kwargs...)
                fcount += fc

                if f2 < f
                    g2, badg2 = NumGrad ? numerical_gradient(fcn, x2; kwargs...) : grad(x2; kwargs...)
                    wall2 = badg2

                    if wall2
                        println("Cliff again. Traversing.")
                        if norm(x2 - x1) < 1e-13
                            f3, x3, badg3, retcode3 = f, x, true, 101
                        else
                            g_cliff = ((f2 - f1) / (norm(x2 - x1)^2)) * (x2 - x1)
                            f3, x3, fc, retcode3 = csminit(fcn, x, f, g_cliff, false, Matrix(I(nx, nx)); kwargs...)
                            fcount += fc
                            g3, badg3 = NumGrad ? numerical_gradient(fcn, x3; kwargs...) : grad(x3; kwargs...)
                        end
                    else
                        f3, x3, badg3, retcode3 = f, x, true, 101
                    end
                else
                    f3, x3, badg3, retcode3 = f, x, true, 101
                end
            else
                f2, f3, badg2, badg3, retcode2, retcode3 = f, f, true, true, 101, 101
            end
        else
            f2, f3, f1, retcode2, retcode3 = f, f, f, retcode1, retcode1
        end

        if !badg && !badg1 && abs(f1 - f) >= crit
            H = bfgsi(H, g1 - g, x1 - x)
        end

        println("Improvement: $(f - f1)")

        if itct >= nit || abs(f - f1) < crit
            done = true
        end

        f, x, g = f1, x1, g1
    end

    return f, x, g, H, itct, fcount, retcode1
end

function numerical_gradient(fcn, x; kwargs...)
    eps = 1e-8
    g = zeros(length(x))
    for i in 1:length(x)
        x_forward = copy(x)
        x_backward = copy(x)
        x_forward[i] += eps
        x_backward[i] -= eps

        g[i] = (fcn(x_forward; kwargs...) - fcn(x_backward; kwargs...)) / (2 * eps)
    end
    return g, false
end

function csminit(fcn, x0, f0, g0, badg, H0; kwargs...)
    ANGLE = 0.005
    THETA = 0.3
    FCHANGE = 1000
    MINLAMB = 1e-9
    MINDFAC = 0.01

    fcount = 0
    lambda = 1.0
    xhat = x0
    fhat = f0
    g = g0
    gnorm = norm(g)

    if gnorm < 1e-12 && !badg
        return fhat, xhat, fcount, 1
    end

    dx = -H0 * g
    dxnorm = norm(dx)
    if dxnorm > 1e12
        dx *= FCHANGE / dxnorm
    end

    dfhat = dot(dx, g0)
    if !badg
        a = -dfhat / (gnorm * dxnorm)
        if a < ANGLE
            dx -= (ANGLE * dxnorm / gnorm + dfhat / (gnorm^2)) * g
            dx *= dxnorm / norm(dx)
            dfhat = dot(dx, g)
        end
    end

    done = false
    factor = 3.0
    shrink = true
    lambda_min = 0.0
    lambda_max = Inf
    lambda_peak = 0.0
    f_peak = f0
    lambdahat = 0.0

    while !done
        dxtest = x0 + lambda * dx
        f = fcn(dxtest; kwargs...)
        fcount += 1

        if f < fhat
            fhat = f
            xhat = dxtest
            lambdahat = lambda
        end

        shrink_signal = (!badg && (f0 - f < max(-THETA * dfhat * lambda, 0))) || (badg && (f0 - f) < 0)
        grow_signal = !badg && (lambda > 0 && (f0 - f > -(1 - THETA) * dfhat * lambda))

        if shrink_signal && (lambda > lambda_peak || lambda < 0)
            if lambda > 0 && (!shrink || lambda / factor <= lambda_peak)
                shrink = true
                factor = factor^0.6
                while lambda / factor <= lambda_peak
                    factor = factor^0.6
                end

                if abs(factor - 1) < MINDFAC
                    return fhat, xhat, fcount, if abs(lambda) < 4.0 2 else 7 end
                end
            end
            lambda = lambda / factor
            if abs(lambda) < MINLAMB
                if lambda > 0 && f0 <= fhat
                    lambda = -lambda * factor^6
                else
                    return fhat, xhat, fcount, if lambda < 0 6 else 3 end
                end
            end
        elseif grow_signal && lambda > 0 || (shrink_signal && (lambda <= lambda_peak && lambda > 0))
            if shrink
                shrink = false
                factor = factor^0.6
                if abs(factor - 1) < MINDFAC
                    return fhat, xhat, fcount, if abs(lambda) < 4.0 4 else 7 end
                end
            end
            if f < f_peak && lambda > 0
                f_peak = f
                lambda_peak = lambda
                if lambda_max <= lambda_peak
                    lambda_max = lambda_peak * factor^2
                end
            end
            lambda = lambda * factor
            if abs(lambda) > 1e20
                return fhat, xhat, fcount, 5
            end
        else
            done = true
            return fhat, xhat, fcount, if factor < 1.2 7 else 0 end
        end
    end

    return fhat, xhat, fcount, 0
end

function bfgsi(H, delta_g, delta_x)
    rho = 1.0 / (dot(delta_g, delta_x))
    V = I(length(delta_x)) - rho * (delta_x * delta_g')
    return V * H * V' + rho * (delta_x * delta_x')
end


bfgsi (generic function with 1 method)

In [357]:
step_fn(x,c)=x>c ? x : Inf

step_fn (generic function with 1 method)

In [315]:
frac(x)=x-floor(x)

frac (generic function with 1 method)

In [394]:
fcn=x->x[1]^2+(x[2]-floor(x[1]))*(x[1]+x[2])+step_fn(x[1],50.0) #Function with discontinuities and nonlinearities
x0=[50.8,0.3];
H0=I(2);
fval=fcn(x0);

fmin, x_min, gmin, Hmin, itct, fcount, retcode1=csminwel(fcn, x0, H0, nothing, 1e-9, 100)



-----------------
Iteration: 0
Function value: 91.7699999999998
x: [50.8, 0.3]
2
Cliff detected. Perturbing search direction.
Improvement: 41.48401736533127
-----------------
Iteration: 1
Function value: 50.28598263466853
x: [50.004059391244084, 0.2789352883661515]
2
Cliff detected. Perturbing search direction.
Improvement: 0.20579427774757164
-----------------
Iteration: 2
Function value: 50.08018835692096
x: [50.000017847609655, 0.28155479649636483]
4
Cliff detected. Perturbing search direction.
Improvement: 0.0009080040435165415
-----------------
Iteration: 3
Function value: 50.07928035287744
x: [50.00000005323721, 0.28156282210216843]
6
Improvement: 0.0


(50.07928035287744, [50.00000005323721, 0.28156282210216843], [51.281551094461975, 0.5631136446027085], [0.7663736756140707 -0.7501425746656414; -0.7501425746656414 1.1473152732912437], 4, 146, 0)

In [371]:
fmin, x_min

(50.07928035287744, [50.00000005323721, 0.28156282210216843])

In [395]:
res=Optim.optimize(fcn,x0)

Optim.minimizer(res)

2-element Vector{Float64}:
 57.000000000615174
  0.0001256793006087301

In [381]:
rosenbrock=x->(12.0-x[1])^2+0.5*((x[2]-x[1]^2)^2) #Rosenbruck function (easy for the algorithm to handle)
x0=[-100.8,0.1];
H0=I(2);
fval=rosenbrock(x0);

fmin, x_min, gmin, Hmin, itct, fcount, retcode1=csminwel(rosenbrock, x0, H0, nothing, 1e-9, 100)



-----------------
Iteration: 0
Function value: 5.16310103858e7
x: [-100.8, 0.1]
0
Improvement: 5.163088295946416e7
-----------------
Iteration: 1
Function value: 127.426335832148
x: [3.2790995610097724, 0.6161996016880061]
0
Improvement: 50.252969046240025
-----------------
Iteration: 2
Function value: 77.17336678590797
x: [3.225187135459662, 10.995171722342741]
0
Improvement: 0.24683342533235475
-----------------
Iteration: 3
Function value: 76.92653336057562
x: [3.229441302852498, 10.341729716224595]
0
Improvement: 0.24310861391994365
-----------------
Iteration: 4
Function value: 76.68342474665567
x: [3.256419490797263, 9.921296999133167]
0
Improvement: 5.513527863525923
-----------------
Iteration: 5
Function value: 71.16989688312975
x: [4.126773467438723, 12.744888590784187]
0
Improvement: 19.02440425892484
-----------------
Iteration: 6
Function value: 52.14549262420491
x: [4.811321537444132, 24.116693633815434]
0
Improvement: 12.691668679596468
-----------------
Iteration: 7
Fun

(5.51293986216348e-12, [11.999999816461864, 143.9999989054484], [-7.981580951315583e-5, 3.310366235954088e-6], [0.5004176271211012 12.002018479356085; 12.002018479356083 289.00029362068767], 21, 49, 0)

In [333]:
fmin,x_min

(5.51293986216348e-12, [11.999999816461864, 143.9999989054484])

In [335]:
res=Optim.optimize(rosenbrock,x0)

Optim.minimizer(res)

 * Status: success

 * Candidate solution
    Final objective value:     6.109127e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    87
    f(x) calls:    166


In [406]:
g=x->exp(abs(x[3]-.85)*(x[1]-.2)^2+x[1]*exp(x[2]*x[3]-.1))

x0=[-0.0,0.0,0.0];
H0=I(3);
fval=g(x0);

fmin, x_min, gmin, Hmin, itct, fcount, retcode1=csminwel(g, x0, H0, nothing, 1e-9, 100)




-----------------
Iteration: 0
Function value: 1.034584606728118
x: [-0.0, 0.0, 0.0]
0
Improvement: 0.0794692677641139
-----------------
Iteration: 1
Function value: 0.955115338964004
x: [-0.19479069838605292, 0.0, 0.013794461869072922]
7
Improvement: 0.50845580293049
-----------------
Iteration: 2
Function value: 0.4466595360335141
x: [-0.9779186892293125, 0.014853134003599958, 0.9155964499776079]
0
Improvement: 0.044571621883696455
-----------------
Iteration: 3
Function value: 0.40208791414981765
x: [-0.9799558263778712, 0.08003193442343741, 0.8240937840939946]
0
Improvement: 0.40182007877984727
-----------------
Iteration: 4
Function value: 0.00026783536997035607
x: [-2.405105347951616, 1.5324146160020762, 0.8875760531619947]
0
Improvement: 0.00017533632519917797
-----------------
Iteration: 5
Function value: 9.249904477117811e-5
x: [-2.483639166162837, 1.6282553904800885, 0.8904888522474387]
0
Improvement: 7.179075860676224e-5
-----------------
Iteration: 6
Function value: 2.07082

(4.1769417258642105e-10, [-3.0312472005964066, 2.296926536282813, 0.9110607960718854], [2.898908277715629e-9, -8.460958322703875e-9, -1.697027026876624e-8], [443422.1488515844 -541475.4338211523 -16675.74374964535; -541475.4338211523 661213.2126834795 20363.27159728642; -16675.743749645353 20363.27159728642 627.201722737248], 15, 39, 0)

In [407]:
res=Optim.optimize(fcn,x0)

Optim.minimizer(res)

3-element Vector{Float64}:
 -0.0
  0.0
  0.0

In [401]:
fmin,x_min

(4.1769417258642105e-10, [-3.0312472005964066, 2.296926536282813, 0.9110607960718854])